<a href="https://colab.research.google.com/github/4GeeksAcademy/Proyecto_Final_Close-Loop-Intelligence/blob/main/Proyecto_Final_Close_Loop_Intelligence_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Idea Integral Ganadora: Close Loop Intelligence
---

1. ***Definición del Problema (El "Pitch" de Negocio)***

    Muchas empresas sufren de dos males: el stockout (perder ventas por no tener producto) o el exceso de inventario (dinero muerto en estanterías que se deprecia). Este proyecto construye un Sistema de Inteligencia de Ciclo Cerrado que predice cuánto se va a vender para decidir exactamente cuánto comprar y qué promocionar para salir del inventario lento.

2. **Arquitectura de Datos (Paso 2 y 3: Ingesta y SQL)**

    Para llegar a las 60,000+ filas, utilizaremos un dataset transaccional histórico.

    Origen: Dataset de Retail (tipo Online Retail II o Instacart).

    Modelo de Datos (SQL): Almacenarás los datos en una base de datos relacional con este esquema:

        Tabla_Ventas: ID_Transaccion, Fecha, ID_Producto, Cantidad, Precio_Unitario, ID_Cliente.

        Tabla_Productos: ID_Producto, Categoria, Costo_Reposicion, Stock_Actual, Punto_Reorden.

    Uso de SQL (Crítico): No solo cargarás los datos. Usarás SQL para crear una Vista Maestra mediante un JOIN. Calcularás el "Lead Time" (tiempo de entrega) y la "Venta Promedio Diaria" usando GROUP BY y funciones agregadas. Esta vista será la que alimentará tu modelo de Machine Learning.

3. **El Cerebro del Proyecto (Paso 4, 5 y 6: EDA y ML)**

    Aquí es donde ocurre la magia. Dividiremos el modelo en dos enfoques:

    *Enfoque de Ventas (Predicción de Demanda):*

    Utilizarás un modelo de Series Temporales o Regresión (como Random Forest Regressor) para predecir cuántas unidades de un producto se venderán en los próximos 15-30 días basándote en la estacionalidad y tendencias históricas.

    *Enfoque de Compras/Inventario (Optimización):*

    Clasificación ABC: Usarás los datos para clasificar productos. Los "A" son los que más dinero generan (nunca pueden faltar), los "C" son los lentos.

    Cálculo de Stock de Seguridad: Aplicarás fórmulas estadísticas para determinar el inventario mínimo necesario.

4. **La Salida en Ventas: "Revenue Optimizer"**

    Para darle ese toque final de ventas que mencionaste, añadiremos un componente de Estrategia Comercial:

    Si el modelo predice una demanda baja para un producto con mucho stock (Exceso), la aplicación sugerirá automáticamente un descuento o promoción de "cross-selling" para forzar la salida de esa mercancía y recuperar flujo de caja.

    No podremos aplicar la estrategia de cross-selling dado que no contamos con stock en el modelo de datos.

5. **Desarrollo de la Web App (Paso 7: La interfaz)**
    
    La aplicación (en Streamlit) tendrá tres pestañas claras para el usuario final:

    *Dashboard Ejecutivo (Ventas):* Gráficos elocuentes que muestran la proyección de ingresos para el próximo mes y el "Top 10" de productos estrella.

    *Módulo de Compras (Reposición):* Una tabla generada por tu modelo que diga: "Debes pedir 500 unidades del Producto X hoy mismo al proveedor Y para no quedar desabastecido el lunes".

    *Módulo de Ofertas (Salida de Inventario):* Una sección que resalte: "¡Alerta! Tienes demasiado stock de este producto. Aplica un 15% de descuento para incentivar la venta". No podremos aplicar la estrategia del modulo de ofertas ya que no contamos con el dato de stock en el dataset.

In [118]:
#Realizamos la instalacion para ejecutar la conexion con Kaggle
!pip install kaggle
!pip install opendatasets

In [119]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import opendatasets as od
import sqlite3
import os
import glob

od.download('https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce')

Skipping, found downloaded files in "./brazilian-ecommerce" (use force=True to force download)


In [120]:
#Configuramos Nombres de Base de Datos
BD = 'BaseSQL.db'
csv_files = glob.glob('brazilian-ecommerce/*.csv')

conexion = sqlite3.connect(BD)

try:
    print(f"Iniciando migración de {len(csv_files)} archivos a la base de datos...")

    for archivo in csv_files:
        # Limpiamos el nombre para la tabla (sin extensión y sin caracteres raros)
        nombre_tabla = os.path.splitext(archivo)[0].replace(" ", "_").lower()

        # Cargamos el archivo a un DataFrame
        df = pd.read_csv(archivo)

        # 3. Guardar en la base de datos
        # index=False evita que se cree una columna extra para el índice de pandas
        df.to_sql(nombre_tabla, conexion, if_exists='replace', index=False)

        print(f"Cargado con éxito: {archivo} -> Tabla: {nombre_tabla}")

    print("\nProceso finalizado. Ahora puedes borrar los .csv originales si lo deseas.")

finally:
    conexion.close()

Iniciando migración de 9 archivos a la base de datos...
Cargado con éxito: brazilian-ecommerce/olist_customers_dataset.csv -> Tabla: brazilian-ecommerce/olist_customers_dataset
Cargado con éxito: brazilian-ecommerce/olist_order_reviews_dataset.csv -> Tabla: brazilian-ecommerce/olist_order_reviews_dataset
Cargado con éxito: brazilian-ecommerce/olist_order_items_dataset.csv -> Tabla: brazilian-ecommerce/olist_order_items_dataset
Cargado con éxito: brazilian-ecommerce/olist_order_payments_dataset.csv -> Tabla: brazilian-ecommerce/olist_order_payments_dataset
Cargado con éxito: brazilian-ecommerce/olist_sellers_dataset.csv -> Tabla: brazilian-ecommerce/olist_sellers_dataset
Cargado con éxito: brazilian-ecommerce/product_category_name_translation.csv -> Tabla: brazilian-ecommerce/product_category_name_translation
Cargado con éxito: brazilian-ecommerce/olist_orders_dataset.csv -> Tabla: brazilian-ecommerce/olist_orders_dataset
Cargado con éxito: brazilian-ecommerce/olist_products_dataset.csv

In [121]:
#Consulta SQL
conexion = sqlite3.connect(BD)
cursor = conexion.cursor()

cursor.execute('SELECT name FROM sqlite_master WHERE type = "table"')
tablas = cursor.fetchall()

for tabla in tablas:
    print(tabla[0])


brazilian-ecommerce/olist_customers_dataset
brazilian-ecommerce/olist_order_reviews_dataset
brazilian-ecommerce/olist_order_items_dataset
brazilian-ecommerce/olist_order_payments_dataset
brazilian-ecommerce/olist_sellers_dataset
brazilian-ecommerce/product_category_name_translation
brazilian-ecommerce/olist_orders_dataset
brazilian-ecommerce/olist_products_dataset
brazilian-ecommerce/olist_geolocation_dataset


In [122]:
# Consulta simple para listar todas las tablas
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Ejecución
df_tablas = pd.read_sql_query(query, conexion)
print(df_tablas)

                                                name
0        brazilian-ecommerce/olist_customers_dataset
1    brazilian-ecommerce/olist_order_reviews_dataset
2      brazilian-ecommerce/olist_order_items_dataset
3   brazilian-ecommerce/olist_order_payments_dataset
4          brazilian-ecommerce/olist_sellers_dataset
5  brazilian-ecommerce/product_category_name_tran...
6           brazilian-ecommerce/olist_orders_dataset
7         brazilian-ecommerce/olist_products_dataset
8      brazilian-ecommerce/olist_geolocation_dataset


In [123]:
#Ejecutamos una consulta simple - DF Order Review

conn = sqlite3.connect(BD)
tables = ['order_reviews', 'products','orders','customers','sellers','order_items','geolocation','order_payments']
dfT = {}

for table in tables:
    query = f'SELECT * FROM "brazilian-ecommerce/olist_{table}_dataset" '
    dfT[table] = pd.read_sql_query(query, conn)
    print(f"Tabla '{table}' cargada con {len(dfT[table])} filas.")



Tabla 'order_reviews' cargada con 99224 filas.
Tabla 'products' cargada con 32951 filas.
Tabla 'orders' cargada con 99441 filas.
Tabla 'customers' cargada con 99441 filas.
Tabla 'sellers' cargada con 3095 filas.
Tabla 'order_items' cargada con 112650 filas.
Tabla 'geolocation' cargada con 1000163 filas.
Tabla 'order_payments' cargada con 103886 filas.


In [124]:
for t in tables:
  print(dfT[t].shape)

(99224, 7)
(32951, 9)
(99441, 8)
(99441, 5)
(3095, 4)
(112650, 7)
(1000163, 5)
(103886, 5)


In [125]:
dfT['customers'].head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [126]:
dfT['order_items'].head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [127]:
dfT['order_payments'].head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [128]:
dfT['orders'].head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [129]:
dfT['products'].head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [130]:
dfT['sellers'].head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [131]:
dfT['geolocation'].head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [132]:
dfT['order_reviews'].head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [133]:
#Prueba de Ejecucion del Master JOUN para las Tablas de Ventas

##Columnas de la primera transformacion para la tabla de ventas: Fecha, ID_Orden, ID_Cliente, ID_Producto, Posicon_Item, Precio_Unitario

query_maestra = """
SELECT
    o.order_purchase_timestamp AS Fecha,
    o.order_id AS ID_Orden,
    o.customer_id AS ID_Cliente,
    i.product_id AS ID_Producto,
    i.order_item_id AS Posicion_Item,
    i.price AS Precio_Unitario,
    i.price AS Precio_Unitario,
    i.freight_value AS Costo_Flete,
    pay.payment_value AS Total_Pago_Orden
FROM "brazilian-ecommerce/olist_orders_dataset" o
JOIN "brazilian-ecommerce/olist_order_items_dataset" i ON o.order_id = i.order_id
JOIN "brazilian-ecommerce/olist_order_payments_dataset" pay ON o.order_id = pay.order_id
LIMIT 1000;
"""

df_ventas = pd.read_sql_query(query_maestra, conn)
display(df_ventas.head())

#Cabe destacar que este DF de Ventas aun no contempla cantidad de items vendidos dentro de una orden. Por tanto, se requieren mas transformaciones

,Fecha,ID_Orden,ID_Cliente,ID_Producto,Posicion_Item,Precio_Unitario,Precio_Unitario,Costo_Flete,Total_Pago_Orden
0,2017-10-02 10:56:33,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,87285b34884572647811a353c7ac498a,1,29.99,29.99,8.72,2.00
1,2017-10-02 10:56:33,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,87285b34884572647811a353c7ac498a,1,29.99,29.99,8.72,18.12
2,2017-10-02 10:56:33,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,87285b34884572647811a353c7ac498a,1,29.99,29.99,8.72,18.59
3,2018-07-24 20:41:37,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,595fac2a385ac33a80bd5114aec74eb8,1,118.70,118.70,22.76,141.46
4,2018-08-08 08:38:49,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,aa4383b373c6aca5d8797843e5594415,1,159.90,159.90,19.22,179.12


In [134]:
# Calculamos la cantidad de productos, el Subtotal de la Orden, y el Total de la Misma

queryV = """
  SELECT
    o.order_id,
    i.product_id,
    i.price AS Precio_Unitario,
    COUNT(i.product_id) AS Cantidad_Productos,
    COUNT(i.product_id) * i.price AS Subtotal_Orden,
    i.freight_value AS Costo_Flete,
    (COUNT(i.product_id) * i.price) + i.freight_value AS Total_Orden,
    SUM(pay.payment_value) AS Total_Pago_Orden

  FROM "brazilian-ecommerce/olist_orders_dataset" o
  JOIN "brazilian-ecommerce/olist_order_items_dataset" i ON o.order_id = i.order_id
  JOIN "brazilian-ecommerce/olist_order_payments_dataset" pay ON o.order_id = pay.order_id
  GROUP BY o.order_id;
    """

consultaV = pd.read_sql_query(queryV, conn)
consultaV.sort_values(by='Cantidad_Productos', ascending=False).head()

,order_id,product_id,Precio_Unitario,Cantidad_Productos,Subtotal_Orden,Costo_Flete,Total_Orden,Total_Pago_Orden
52745,895ab968e7bb0d5659d16cd74cd1650c,5ddab10d5e0a23acb99acf56b62b3276,83.80,63,5279.40,5.12,5284.52,483.96
98246,fedcd9f7ccdc8cba3a18defedd1a5547,0554911df28fda9fd668ce5ba5949695,79.99,38,3039.62,22.88,3062.50,411.48
96501,fa65dad1b0e818e3ccc5cb0e39231352,1aecdb5fa3add74e385f25c6c527a462,392.55,29,11383.95,65.44,11449.39,457.99
78975,ccf804e764ed5650cd8759557269dc13,8d37ee446981d3790967d0268d6cfc81,49.99,26,1299.74,12.69,1312.43,62.68
76421,c6492b842ac190db807c15aff21a7dd6,0449db5eede617c5fd413071d582f038,189.90,24,4557.60,22.80,4580.40,850.80


In [135]:
# Averiguamos la cantidad de pagos que se

pago_q = """
  SELECT
    pay.order_id,
    SUM(pay.payment_value) AS Total_Pago_Orden,
    COUNT(pay.payment_type) AS Cantidad_Pagos
  FROM "brazilian-ecommerce/olist_order_payments_dataset" pay
  GROUP BY pay.order_id;
"""

consulta_pago = pd.read_sql_query(pago_q, conn)
consulta_pago.sort_values(by='Total_Pago_Orden', ascending=False).head()


,order_id,Total_Pago_Orden,Cantidad_Pagos
1471,03caa2c082116e1d31e67e9ae3700499,13664.08,1
44830,736e1922ae60d0d6a89247b851902527,7274.88,1
3156,0812eb902a67711a1cb742b3cdaa65ae,6929.31,1
99071,fefacc66af859508bf1a7934eab1e97f,6922.21,1
95186,f5136e38d1a14a4dbd87dff67da82701,6726.66,1


In [136]:
pagosdf = pd.DataFrame(consulta_pago)
pagosdf[pagosdf['order_id'] == '0008288aa423d2a3f00fcb17cd7d8719']

,order_id,Total_Pago_Orden,Cantidad_Pagos
13,0008288aa423d2a3f00fcb17cd7d8719,126.54,1


In [137]:
r = pd.DataFrame(consultaV)
r.loc[r['order_id'] == '03caa2c082116e1d31e67e9ae3700499']

,order_id,product_id,Precio_Unitario,Cantidad_Productos,Subtotal_Orden,Costo_Flete,Total_Orden,Total_Pago_Orden
1455,03caa2c082116e1d31e67e9ae3700499,5769ef0a239114ac3a854af00df129e4,1680.0,8,13440.0,28.01,13468.01,109312.64


In [138]:
#Rehacemos el query final para la obtencion del DataFrame de Ventas

query_final = """
  WITH Total_Pagos AS (
    SELECT
      pay.order_id,
      SUM(pay.payment_value) AS Total_Pago_Orden
    FROM "brazilian-ecommerce/olist_order_payments_dataset" pay
    GROUP BY pay.order_id
  )
SELECT
    o.order_purchase_timestamp AS "Fecha",
    o.order_id AS "Orden ID",
    o.customer_id AS "Cliente ID",
    c.customer_unique_id AS "Nombre del Cliente",
    c.customer_city AS "Ciudad Cliente",
    s.seller_id AS "Vendedor ID",
    s.seller_city AS "Ciudad Vendedor",
    i.product_id AS "Producto ID",
    p.product_category_name AS "Categoria Producto",
    p.product_weight_g AS "Peso Producto",
    p.product_length_cm AS "Longitud Producto",
    p.product_height_cm AS "Altura Producto",
    p.product_width_cm AS "Ancho Producto",
    p.product_photos_qty AS "Cantidad Fotos Producto",
    i.price AS "Precio Unitario",
    COUNT(i.product_id) AS "Cantidad",
    COUNT(i.product_id) * i.price AS "Subtotal Orden",
    i.freight_value AS "Costo Flete",
    (COUNT(i.product_id) * i.price) + i.freight_value AS "Total Orden",
    SUM(pay.payment_value) AS "Total Pago Orden",
    (o.order_purchase_timestamp - order_estimated_delivery_date) AS "Tiempo Entrega Estimado",
    o.order_status AS "Estado Orden",
    r.review_score AS "Calificacion"


  FROM "brazilian-ecommerce/olist_orders_dataset" o
  JOIN "brazilian-ecommerce/olist_order_items_dataset" i ON o.order_id = i.order_id
  JOIN "brazilian-ecommerce/olist_order_payments_dataset" pay ON o.order_id = pay.order_id
  JOIN "brazilian-ecommerce/olist_customers_dataset" c ON o.customer_id = c.customer_id -- Agregamos el join para la tabla de clientes
  JOIN "brazilian-ecommerce/olist_sellers_dataset" s ON i.seller_id = s.seller_id
  JOIN "brazilian-ecommerce/olist_products_dataset" p ON i.product_id = p.product_id
  LEFT JOIN "brazilian-ecommerce/olist_order_reviews_dataset" r ON o.order_id = r.order_id
  GROUP BY o.order_id; -- Es posible que necesitemos ajustar el GROUP BY más adelante.
  """

Ventas_DF = pd.read_sql_query(query_final, conn)
Ventas_DF.sort_values(by='Cantidad', ascending=False).head()

,Fecha,Orden ID,Cliente ID,Nombre del Cliente,Ciudad Cliente,Vendedor ID,Ciudad Vendedor,Producto ID,Categoria Producto,Peso Producto,...,Cantidad Fotos Producto,Precio Unitario,Cantidad,Subtotal Orden,Costo Flete,Total Orden,Total Pago Orden,Tiempo Entrega Estimado,Estado Orden,Calificacion
52745,2017-08-08 20:26:31,895ab968e7bb0d5659d16cd74cd1650c,270c23a11d024a44c896d1894b261a83,9a736b248f67d166d2fbb006bcb877c3,sao paulo,3d0cd21d41671c46f82cd11176bf7277,joinville,5ddab10d5e0a23acb99acf56b62b3276,utilidades_domesticas,450.0,...,8.0,83.80,63,5279.40,5.12,5284.52,483.96,0,delivered,5.0
98246,2017-09-23 14:56:45,fedcd9f7ccdc8cba3a18defedd1a5547,13aa59158da63ba0e93ec6ac2c07aacb,6fbc7cdadbb522125f4b27ae9dee4060,rio de janeiro,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,0554911df28fda9fd668ce5ba5949695,moveis_escritorio,5200.0,...,1.0,79.99,38,3039.62,22.88,3062.50,411.48,0,delivered,5.0
96501,2017-04-20 12:45:34,fa65dad1b0e818e3ccc5cb0e39231352,9af2372a1e49340278e7c1ef8d749f34,8af7ac63b2efbcbd88e5b11505e8098a,cuiaba,06532f10282704ef4c69168b914b77be,paincandu,1aecdb5fa3add74e385f25c6c527a462,ferramentas_jardim,16400.0,...,1.0,392.55,29,11383.95,65.44,11449.39,457.99,0,shipped,1.0
78975,2017-06-07 12:05:10,ccf804e764ed5650cd8759557269dc13,92cd3ec6e2d643d4ebd0e3d6238f69e2,569aa12b73b5f7edeaa6f2a01603e381,sao paulo,1900267e848ceeba8fa32d80c1a5f5a8,ibitinga,8d37ee446981d3790967d0268d6cfc81,cama_mesa_banho,1100.0,...,1.0,49.99,26,1299.74,12.69,1312.43,62.68,0,delivered,5.0
40301,2017-07-07 14:55:43,68986e4324f6a21481df4e6e89abcf01,86cc80fef09f7f39df4b0dbce48e81cb,db1af3fd6b23ac3873ef02619d548f9c,itaqui,dbc22125167c298ef99da25668e1011f,borda da mata,2b939dc9b176d7fa21594d588815d4a4,malas_acessorios,1300.0,...,1.0,102.90,24,2469.60,8.73,2478.33,658.44,0,delivered,1.0


In [139]:
n = pd.DataFrame(Ventas_DF)
n.loc[n['Orden ID'] == '03caa2c082116e1d31e67e9ae3700499' ]


,Fecha,Orden ID,Cliente ID,Nombre del Cliente,Ciudad Cliente,Vendedor ID,Ciudad Vendedor,Producto ID,Categoria Producto,Peso Producto,...,Cantidad Fotos Producto,Precio Unitario,Cantidad,Subtotal Orden,Costo Flete,Total Orden,Total Pago Orden,Tiempo Entrega Estimado,Estado Orden,Calificacion
1455,2017-09-29 15:24:52,03caa2c082116e1d31e67e9ae3700499,1617b1357756262bfa56ab541c47bc16,0a0a92112bd4c708ca5fde585afaa872,rio de janeiro,b37c4c02bda3161a7546a4e6d222d5b2,sao paulo,5769ef0a239114ac3a854af00df129e4,telefonia_fixa,750.0,...,3.0,1680.0,8,13440.0,28.01,13468.01,109312.64,0,delivered,1.0


In [140]:
n.shape

(98665, 23)

In [141]:
pagosdf[pagosdf['order_id'] == '016726239765c18f66826453f39c64e3']

,order_id,Total_Pago_Orden,Cantidad_Pagos
533,016726239765c18f66826453f39c64e3,265.77,1


In [142]:
n.loc[n['Orden ID']== '016726239765c18f66826453f39c64e3']

,Fecha,Orden ID,Cliente ID,Nombre del Cliente,Ciudad Cliente,Vendedor ID,Ciudad Vendedor,Producto ID,Categoria Producto,Peso Producto,...,Cantidad Fotos Producto,Precio Unitario,Cantidad,Subtotal Orden,Costo Flete,Total Orden,Total Pago Orden,Tiempo Entrega Estimado,Estado Orden,Calificacion
525,2017-06-14 09:16:44,016726239765c18f66826453f39c64e3,6204061101097bad0b25b3c4fce9ee8d,6724236615afbc343645dacfa693fae0,sao paulo,37be5a7c751166fbc5f8ccba4119e043,sao paulo,c6ea63369936552872ae890c82175b4b,cama_mesa_banho,2900.0,...,4.0,219.99,1,219.99,15.14,235.13,265.77,0,delivered,5.0


In [143]:


n['Diferencia de Pago'] = n['Total Pago Orden'] - n['Total Orden']
n.loc[n['Diferencia de Pago'] > 10].sort_values(by='Cantidad', ascending=False)

,Fecha,Orden ID,Cliente ID,Nombre del Cliente,Ciudad Cliente,Vendedor ID,Ciudad Vendedor,Producto ID,Categoria Producto,Peso Producto,...,Precio Unitario,Cantidad,Subtotal Orden,Costo Flete,Total Orden,Total Pago Orden,Tiempo Entrega Estimado,Estado Orden,Calificacion,Diferencia de Pago
62660,2018-05-12 12:28:58,a3725dfe487d359b5be08cac48b64ec5,d22f25a9fadfb1abbc2e29395b1239f4,5419a7c9b86a43d8140e2939cd2c2f7e,sinop,9f505651f4a6abe901a56cdc21508025,sao paulo,4e53a453045707bbc5febcf5f32097ac,informatica_acessorios,250.0,...,14.99,24,359.76,22.06,381.82,1333.80,0,delivered,3.0,951.98
34716,2017-10-17 13:06:29,5a3b1c29a49756e75f1ef513383c0c12,be1c4e52bb71e0c54b11a26b8e8d59f2,d97b3cfb22b0d6b25ac9ed4e9c2d481b,sao paulo,d2374cbcbb3ca4ab1086534108cc3ab7,ibitinga,17d8b8e9c21ec0fc5b33724e32fe422e,cama_mesa_banho,1000.0,...,58.90,22,1295.80,2.66,1298.46,18786.90,0,delivered,1.0,17488.44
10831,2018-01-27 11:28:32,1c11d0f4353b31ac3417fbfa5f0f2a8a,50920f8cd0681fd86ebe93670c8fe52e,41a3b256cc497dc952a815b848345cbc,ijui,c7fdb77fdbff3c41981bc52f787e959e,santo andre,61b6e5d2e3ee58d2b341b8ef1a652b26,cama_mesa_banho,750.0,...,56.00,21,1176.00,12.77,1188.77,2951.83,0,delivered,1.0,1763.06
50137,2017-07-16 18:19:25,8272b63d03f5f79c56e9e4120aec44ef,fc3d1daec319d62d49bfb5e1f83123e9,4546caea018ad8c692964e3382debd19,sao paulo,2709af9587499e95e803a6498a5a56e9,sao paulo,05b515fdc76e888aada3c6d66c201dff,beleza_saude,800.0,...,1.20,21,25.20,7.89,33.09,4118.31,0,delivered,1.0,4085.22
65715,2017-08-23 09:22:34,ab14fdcfbe524636d65ee38360e22ce8,bd5d39761aa56689a265d95d8d32b8be,698e1cf81d01a3d389d96145f7fa6df8,goiania,ce27a3cc3c8cc1ea79d11e561e9bebb6,sao paulo,9571759451b1d780ee7c15012ea109d4,automotivo,1400.0,...,98.70,20,1974.00,14.44,1988.44,45256.00,0,delivered,NaN,43267.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8257,2017-08-08 13:44:39,156ba06ab4556cd19613dca1cefa9f7a,cb196971e6f67237a41f2fe6235bbaf4,ed0ab782b03b402c0cafde88067ff9d2,mineiros,4e922959ae960d389249c378d1c939f5,jacarei,7b25acee1c4b91200a7e8e31bb977a06,utilidades_domesticas,1675.0,...,69.90,1,69.90,17.74,87.64,101.22,0,delivered,4.0,13.58
6474,2018-02-20 01:39:23,10a6730b0b333e2b017dd139a0530f19,ec9941bfae382794f75c237d60273a42,d747d5d50ec718b9ce0c27d8b44cd617,palmares,2e90cb1677d35cfe24eef47d441b7c87,sao paulo,93c902b021a9e594f658ab1b0351602a,eletronicos,1250.0,...,97.90,1,97.90,29.09,126.99,143.53,0,delivered,3.0,16.54
2374,2017-04-21 13:18:31,061c97a9bd7911f9d2031e1b820c344f,ddc1e728d2de6666c8a01a42901d36c3,e411f7646017f3ade17f5db10505736b,junqueiropolis,522620dcb18a6b31cd7bdf73665113a9,cascavel,a70f58bd49225ad1d9f199d3766385a8,informatica_acessorios,500.0,...,299.00,1,299.00,16.26,315.26,329.80,0,delivered,5.0,14.54
1123,2018-05-29 16:04:49,02f4dd90ba0feb8ec394cac05862d2b5,df6d93efba53a613bf2365f7945febb0,9484cca8fd716e613bc1a0bb17ed0172,humaita,e0eabded302882513ced4ea3eb0c7059,campinas,f3c62fde9d2e2acf6c7c46648571c070,consoles_games,150.0,...,79.90,1,79.90,51.06,130.96,141.65,0,delivered,5.0,10.69


In [144]:
#Rehacemos nuevamente el query final para la obtencion del DataFrame de Ventas

query_final = """
  WITH Total_Pagos AS (
    SELECT
      pay.order_id,
      SUM(pay.payment_value) AS Total_Pago_Orden
    FROM "brazilian-ecommerce/olist_order_payments_dataset" pay
    GROUP BY pay.order_id
  )
SELECT
    o.order_purchase_timestamp AS "Fecha",
    o.order_id AS "Orden ID",
    o.customer_id AS "Cliente ID",
    c.customer_unique_id AS "Nombre del Cliente",
    c.customer_state AS "Ciudad Cliente",
    s.seller_id AS "Vendedor ID",
    s.seller_city AS "Ciudad Vendedor",
    i.product_id AS "Producto ID",
    p.product_category_name AS "Categoria Producto",
    p.product_weight_g AS "Peso Producto g",
    (p.product_length_cm * p.product_height_cm * p.product_width_cm)  AS "Volumen Producto cm3",
    p.product_photos_qty AS "Fotos Producto",
    i.price AS "Precio Unitario",
    i.order_item_id AS "Cantidad",
    (i.order_item_id * i.price) AS "Subtotal Orden",
    i.freight_value AS "Costo Flete",
    (i.order_item_id * i.price) + i.freight_value AS "Total Orden",
    tp.Total_Pago_Orden AS "Total Pago Orden",
    (JULIANDAY(o.order_estimated_delivery_date) - JULIANDAY(o.order_purchase_timestamp)) AS "Tiempo Entrega Estimado (Dias)",
    o.order_status AS "Estado Orden",
    r.review_score AS "Calificacion"


  FROM "brazilian-ecommerce/olist_orders_dataset" o
  JOIN "brazilian-ecommerce/olist_order_items_dataset" i ON o.order_id = i.order_id
  JOIN "brazilian-ecommerce/olist_order_payments_dataset" pay ON o.order_id = pay.order_id
  JOIN "brazilian-ecommerce/olist_customers_dataset" c ON o.customer_id = c.customer_id
  JOIN "brazilian-ecommerce/olist_sellers_dataset" s ON i.seller_id = s.seller_id
  JOIN "brazilian-ecommerce/olist_products_dataset" p ON i.product_id = p.product_id
  LEFT JOIN "brazilian-ecommerce/olist_order_reviews_dataset" r ON o.order_id = r.order_id
  JOIN Total_Pagos tp ON o.order_id = tp.order_id
  GROUP BY o.order_id, i.product_id;
  """

Ventas_DF = pd.read_sql_query(query_final, conn)
Ventas_DF.sort_values(by='Cantidad', ascending=False).head()

,Fecha,Orden ID,Cliente ID,Nombre del Cliente,Ciudad Cliente,Vendedor ID,Ciudad Vendedor,Producto ID,Categoria Producto,Peso Producto g,...,Fotos Producto,Precio Unitario,Cantidad,Subtotal Orden,Costo Flete,Total Orden,Total Pago Orden,Tiempo Entrega Estimado (Dias),Estado Orden,Calificacion
52029,2017-07-16 18:19:25,8272b63d03f5f79c56e9e4120aec44ef,fc3d1daec319d62d49bfb5e1f83123e9,4546caea018ad8c692964e3382debd19,SP,2709af9587499e95e803a6498a5a56e9,sao paulo,79ce45dbc2ea29b22b5a261bbb7b7ee7,beleza_saude,1000.0,...,2.0,7.8,21,163.8,6.57,170.37,196.11,11.236516,delivered,1.0
39714,2018-04-17 21:08:59,637617b3ffe9e2f7a2411243829226d0,daf15f1b940cc6a72ba558f093dc00dd,37bc3d463e2a0024012a7fa587597a3c,SC,1f50f920176fa81dab994f9023523100,sao jose do rio preto,bf44071ef18f5c9ded039681c68b1996,bebes,6050.0,...,2.0,79.9,10,799.0,24.02,823.02,1246.97,29.118762,delivered,4.0
80870,2018-08-12 02:11:20,ca3625898fbd48669d50701aba51cd5f,0d861a5e4dd6a9079d89e1330848f0ab,c8ed31310fc440a3f8031b177f9842c3,SP,0b35c634521043bf4b47e21547b99ab5,teixeira soares,309dd69eb83cea38c51709d62befe1a4,construcao_ferramentas_construcao,2150.0,...,1.0,56.0,9,504.0,3.68,507.68,1157.28,31.908796,delivered,3.0
80869,2018-08-12 02:11:20,ca3625898fbd48669d50701aba51cd5f,0d861a5e4dd6a9079d89e1330848f0ab,c8ed31310fc440a3f8031b177f9842c3,SP,0b35c634521043bf4b47e21547b99ab5,teixeira soares,21b524c4c060169fa75ccf08c7da4627,construcao_ferramentas_construcao,3350.0,...,1.0,63.7,8,509.6,0.15,509.75,1157.28,31.908796,delivered,3.0
36029,2017-10-17 13:06:29,5a3b1c29a49756e75f1ef513383c0c12,be1c4e52bb71e0c54b11a26b8e8d59f2,d97b3cfb22b0d6b25ac9ed4e9c2d481b,SP,d2374cbcbb3ca4ab1086534108cc3ab7,ibitinga,e95fb3a80ca294f39de925b6428f3a32,cama_mesa_banho,1300.0,...,1.0,78.9,8,631.2,7.98,639.18,853.95,16.453831,delivered,1.0


In [145]:
Ventas_DF.shape

(102424, 21)

In [146]:
# Creamos la variable sintetica de Tiempo de Reposicion de un Producto basado en la categoria
categorias = list((Ventas_DF['Categoria Producto'].unique()))
np.random.seed(42)
lead_time_rep = {c: np.random.randint(1,91) for c in categorias}
lead_time_rep





{'cool_stuff': 52,
 'pet_shop': 15,
 'moveis_decoracao': 72,
 'perfumaria': 61,
 'ferramentas_jardim': 21,
 'utilidades_domesticas': 83,
 'telefonia': 87,
 'beleza_saude': 75,
 'livros_tecnicos': 75,
 'fashion_bolsas_e_acessorios': 88,
 'cama_mesa_banho': 24,
 'esporte_lazer': 3,
 'consoles_games': 22,
 'moveis_escritorio': 53,
 'malas_acessorios': 2,
 'alimentos': 88,
 'agro_industria_e_comercio': 30,
 'eletronicos': 38,
 'informatica_acessorios': 2,
 'construcao_ferramentas_construcao': 64,
 'audio': 60,
 'bebes': 21,
 'construcao_ferramentas_iluminacao': 33,
 'brinquedos': 76,
 'papelaria': 58,
 'industria_comercio_e_negocios': 22,
 'relogios_presentes': 89,
 'automotivo': 49,
 None: 59,
 'eletrodomesticos': 42,
 'moveis_cozinha_area_de_servico_jantar_e_jardim': 60,
 'climatizacao': 80,
 'casa_conforto': 15,
 'telefonia_fixa': 62,
 'portateis_casa_forno_e_cafe': 62,
 'fraldas_higiene': 47,
 'sinalizacao_e_seguranca': 62,
 'instrumentos_musicais': 51,
 'eletroportateis': 55,
 'constr

In [147]:
#Mapeamos el Tiempo de Reposicion por Categoria de Productos en el DF principal
Ventas_DF['Tiempo de Reposicion'] = Ventas_DF['Categoria Producto'].map(lead_time_rep)
display(Ventas_DF[['Producto ID', 'Categoria Producto', 'Tiempo de Reposicion']].head())


,Producto ID,Categoria Producto,Tiempo de Reposicion
0,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,52
1,e5f2d52b802189ee658865ca93d83a8f,pet_shop,15
2,c777355d18b72b67abbeef9df44fd0fd,moveis_decoracao,72
3,7634da152a4610f1595efa32f14722fc,perfumaria,61
4,ac6c3623068f30de03045865e4e10089,ferramentas_jardim,21


In [148]:
#Verificamos si tenemos valores nulos
Ventas_DF.isnull().sum()

,0
Fecha,0
Orden ID,0
Cliente ID,0
Nombre del Cliente,0
Ciudad Cliente,0
Vendedor ID,0
Ciudad Vendedor,0
Producto ID,0
Categoria Producto,1460
Peso Producto g,16


In [149]:
t = Ventas_DF.loc[Ventas_DF['Categoria Producto'].isnull()]
t[['Categoria Producto','Tiempo de Reposicion']]

,Categoria Producto,Tiempo de Reposicion
113,None,59
115,None,59
122,None,59
128,None,59
154,None,59
...,...,...
102103,None,59
102129,None,59
102145,None,59
102230,None,59


In [150]:
Ventas_DF.dropna(inplace=True)

In [151]:
Ventas_DF.isnull().sum()

,0
Fecha,0
Orden ID,0
Cliente ID,0
Nombre del Cliente,0
Ciudad Cliente,0
Vendedor ID,0
Ciudad Vendedor,0
Producto ID,0
Categoria Producto,0
Peso Producto g,0


In [152]:
#Calculo del Stock Minimo Estadistico
products = Ventas_DF.groupby('Producto ID').agg(
    Cantidad_Media = ('Cantidad', 'mean'),
    Cantidad_Desviacion = ('Cantidad', 'std'),
    Tiempo_de_Reposicion = ('Tiempo de Reposicion', 'first')
).reset_index()

products.head()

,Producto ID,Cantidad_Media,Cantidad_Desviacion,Tiempo_de_Reposicion
0,00066f42aeeb9f3007548bb9d3f33c38,1.0,NaN,61
1,00088930e925c41fd95ebfe695fd2655,1.0,NaN,49
2,0009406fd7479715e4bef61dd91f2462,1.0,NaN,24
3,000b8f95fcb9e0096488278317764d19,1.0,0.0,83
4,000d9be29b5207b54e86aa1b1ac54872,1.0,NaN,89


In [153]:
products['Cantidad_Desviacion'].fillna(0, inplace=True)
products.head()

/tmp/ipython-input-716955406.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  products['Cantidad_Desviacion'].fillna(0, inplace=True)


,Producto ID,Cantidad_Media,Cantidad_Desviacion,Tiempo_de_Reposicion
0,00066f42aeeb9f3007548bb9d3f33c38,1.0,0.0,61
1,00088930e925c41fd95ebfe695fd2655,1.0,0.0,49
2,0009406fd7479715e4bef61dd91f2462,1.0,0.0,24
3,000b8f95fcb9e0096488278317764d19,1.0,0.0,83
4,000d9be29b5207b54e86aa1b1ac54872,1.0,0.0,89


In [154]:
# Definicion de Stock Minimo, Stock de Seguridad y Parametros Asociados:

# Nivel de Servicio o Intervalos de Confianza (Logistica Avanzada)
Z_dict = {
    'A': 2.33, #Producto de Alta Rotacion 99% de Confianza
    'B': 1.645, #Producto de Media Rotacion 95% de Confianza
    'C': 1.282   #Producto de Baja Rotacion 80% de Confianza
}


# Se iniciara el calculo del Stock Minimo con un intervalo de Confianza estandar de 95% para luego ajustarlo segun sea la Clasificacion del Producto con Z_dict

#Stock de Seguridad y Stock Minimo
z = Z_dict['B']
products['Stock de Seguridad'] = z * products['Cantidad_Desviacion']* np.sqrt(products['Tiempo_de_Reposicion'])
products['Stock Minimo'] = (products['Cantidad_Media']* products['Tiempo_de_Reposicion']) + products['Stock de Seguridad']

products.sort_values('Cantidad_Media', ascending= False).head()



,Producto ID,Cantidad_Media,Cantidad_Desviacion,Tiempo_de_Reposicion,Stock de Seguridad,Stock Minimo
15440,79ce45dbc2ea29b22b5a261bbb7b7ee7,21.0,0.0,75,0.0,1575.0
24010,bf44071ef18f5c9ded039681c68b1996,10.0,0.0,21,0.0,210.0
26247,d05bb6074b45c20f7a516c239c0a1c43,7.0,0.0,15,0.0,105.0
12330,61b6e5d2e3ee58d2b341b8ef1a652b26,7.0,0.0,24,0.0,168.0
26490,d25bbb182b51d1594fe3b54a27ae143e,7.0,0.0,24,0.0,168.0


In [155]:
products.shape

(32183, 6)

In [158]:
#Clasificacion de Producto por Volumen de Ventas en Cantidad basado en el pareto de las ventas

products = products.sort_values('Cantidad_Media', ascending=False)

#Porcentaje Acumulado sobre el total de unidades vendidas
unidades_vendidas = products['Cantidad_Media'].sum()
products['Porcentaje Acumulado'] = products['Cantidad_Media'].cumsum() / unidades_vendidas

#ClasificacionABC (Variable Objetivo)
products['ClasificacionABC'] = products['Porcentaje Acumulado'].apply(lambda x: 'A' if x <= 0.80 else('B' if x <= 0.95 else 'C'))

products = products.replace({'Cantidad_Media': 'Cantidad Media Vendida', 'Cantidad_Desviacion': 'Desviacion de la Cantidad Media Vendida', 'Clasificacion ABC': 'Clasificacion ABC'}, inplace=True)
products.head()

AttributeError: 'NoneType' object has no attribute 'sort_values'